# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [55]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [58]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1 #ejectividad del modelo

0.9555555555555556

### Actividad 1:

In [65]:
n=5
n_arboles=np.random.randint(5, 51, n)
profundidad=np.random.randint(2,11, n)

lista_f1 = []
for n_arb, prof in zip(n_arboles, profundidad) :
    modelo, f1 = Bosque.RegresionBosque(X, y, n_arb, prof) #x
    lista_f1.append(f1)
lista_f1 #y

[0.9444444444444444,
 0.9555555555555556,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556]

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [79]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

n_arb_vec = n_arboles.reshape([-1, 1])
prof_vec = profundidad.reshape([-1, 1])
X_hiperparam = np.hstack((n_arb_vec, prof_vec))


kernel=1.0*RBF(length_scale=1)
gp=GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(X_hiperparam, lista_f1)

x_arb = np.linspace(5, 50, 1000).reshape([-1, 1])
x_prof = np.linspace(2, 10, 1000).reshape([-1, 1])
x_h = np.hstack((x_arb, x_prof))
y_pred, y_std = gp.predict(x_h, return_std=True)
y_pred_low=y_pred-1.96*y_std
y_pred_high=y_pred+1.96*y_std

In [73]:
x_h

array([[ 5.        ,  2.        ],
       [ 5.04504505,  2.00800801],
       [ 5.09009009,  2.01601602],
       ...,
       [49.90990991,  9.98398398],
       [49.95495495,  9.99199199],
       [50.        , 10.        ]])

In [81]:
for i in range(10):
    kernel=1.0*RBF(length_scale=1)
    gp=GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(x_h, lista_f1)
    x_arb = np.linspace(5, 50, 1000).reshape([-1, 1])
    x_prof = np.linspace(2, 10, 1000).reshape([-1, 1])
    y_pred, y_std = gp.predict(x_h, return_std=True)
    y_pred_low=y_pred-1.96*y_std
    y_pred_high=y_pred+1.96*y_std
    i_next=np.argmax(y_pred_high-y_pred_low)
    X=np.vstack((X, x[i_next]))
    Y=np.vstack((Y, f(x[i_next]))) 




ValueError: Found input variables with inconsistent numbers of samples: [1000, 5]

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.